In [11]:
import warnings
from math import comb

import numpy as np
import pandas as pd
from scipy.stats import bernoulli, logistic

warnings.filterwarnings("ignore")


In [12]:
# Pooling====
# GROUP TESTING
# CREATED: Eli P. Fenichel, Yale University
# LAST UPDATED: July 15, 2020
# Probability of a groups testing positive accounting for sensitivity of test
# with prevalence m, group size = group, population = pop


def estimate_pooling_prob_of_pos_groups(group_size, m, pop, se, s_loss):
    shat = se - group_size * s_loss
    p_pos = 1 - (1 - shat * m) ** group_size  # shat es la sensibilidad,
    # p.pos: Probabilidad que un grupo sea positivo
    # (1-prevalencia): Probabilidad que se salga negativo una persona
    # (1-prevalencia)^tamano_grupo: Probabilidad que todos sean negativos en el grupo
    return p_pos


In [13]:
# Group results
def estimate_pooling_tests(group_size, p, pop, se, s_loss, sp):
    p_pos_pool = estimate_pooling_prob_of_pos_groups(
        group_size, p, pop, se, s_loss)
    p_pos_test = se * p + (1 - sp) * (1 - p)
    # number of groups
    n_group = int(np.round(pop / group_size))
    pos_groups = np.sum(bernoulli.rvs(p_pos_pool, size=n_group))
    neg_groups = n_group - pos_groups  # cambio

    # number of people
    neg_peop_group = neg_groups * group_size
    pos_peop_group = pos_groups * group_size

    pos_peop = pos_peop_group * p_pos_test

    n_groups_pooling = pos_groups + neg_groups
    total_tests_pooling = n_groups_pooling + pos_peop
    test_per_person_pooling = total_tests_pooling / pop

    # the expected number of people in negative groups that are actually
    # false positives with group size = group.size, prevalence p, sensitivity s,
    # in a population p.

    shat = se - group_size * s_loss
    t_prob = (1 - shat) * p + (1 - p)
    expected_number_missed = np.zeros(group_size - 1)
    for x in range(group_size - 1):
        expected_number_missed[x] = (
            comb(group_size, group_size - (x + 1))
            * ((1 - p) / t_prob) ** (group_size - (x + 1))
            * ((1 - shat) * p / t_prob) ** (x + 1)
        )

    exp_m = np.matmul(range(1, group_size), expected_number_missed)
    gm = exp_m * neg_groups

    res = pd.DataFrame(
        {
            "p_pos_pool": [p_pos_pool],
            "p_pos_test": [p_pos_test],
            "pos_groups": [pos_groups],
            "neg_groups": [neg_groups],
            "pos_peop_group": [pos_peop_group],
            "pos_peop": [pos_peop],
            "neg_peop_group": [neg_peop_group],
            "n_groups_pooling": [n_groups_pooling],
            "n_undetected_pos": [gm],
            "undetected_groups": [exp_m],
            "total_tests_pooling": [total_tests_pooling],
            "number_test_per_person_pooling": [test_per_person_pooling],
        }
    )
    return res


In [14]:
def covid_testing_strategies(
    prevalence,
    sens_model,
    spec_model,
    sens_ag_test,
    spec_ag_test,
    sens_pcr_test,
    spec_pcr_test,
    sens_lamp_test,
    spec_lamp_test,
    people,
    cost_pcr,
    cost_antigen,
    cost_lamp,
    group_size,
    strategy,
    budget,
):
    df_out = pd.DataFrame({})
    sens = sens_model
    spec = spec_model
    # for sens in sens_model:
    #     for spec in spec_model:

    # Probability that an a test gives a positive result
    # Antigen
    prob_res_pos_ag = sens_ag_test * prevalence + \
        (1 - spec_ag_test) * (1 - prevalence)
    # LAMP
    prob_res_pos_lamp = sens_lamp_test * prevalence + (1 - spec_lamp_test) * (
        1 - prevalence
    )
    # PCR
    prob_res_pos_pcr = sens_pcr_test * prevalence + (1 - spec_pcr_test) * (
        1 - prevalence
    )
    # Probability that a test gives a negative result
    prob_res_neg_ag = 1 - prob_res_pos_ag
    prob_res_neg_lamp = 1 - prob_res_pos_lamp
    prob_res_neg_pcr = 1 - prob_res_pos_pcr

    # Probability model gives a low risk classification
    # prob_model_low_risk = (1 - sens) * prevalence +
    #   spec * (1 - prevalence)
    # prob_model_high_risk = 1 - prob_model_low_risk

    prob_model_high_risk_ag = sens * \
        prob_res_pos_ag + (1 - spec) * prob_res_neg_ag
    # prob_model_high_risk_ag = 1 - prob_model_low_risk_ag

    prob_model_high_risk_pcr = sens * \
        prob_res_pos_pcr + (1 - spec) * prob_res_neg_pcr
    # prob_model_high_risk_pcr = 1 - prob_model_low_risk_pcr

    q_model_high_risk_ag = logistic.ppf(prob_model_high_risk_ag)
    q_model_high_risk_pcr = logistic.ppf(prob_model_high_risk_pcr)

    prob_model_high_risk = logistic.cdf(
        0.5 * (q_model_high_risk_ag + q_model_high_risk_pcr)
    )
    prob_model_low_risk = 1 - prob_model_high_risk

    # PPV y NPV of model
    npv_model_ag = spec * prob_res_neg_ag / prob_model_low_risk
    npv_model_pcr = spec * prob_res_neg_pcr / prob_model_low_risk

    q_npv_model_low_risk_ag = logistic.ppf(npv_model_ag)
    q_npv_model_low_risk_pcr = logistic.ppf(npv_model_pcr)

    npv_model = logistic.cdf(
        0.5 * (q_npv_model_low_risk_ag + q_npv_model_low_risk_pcr))
    #
    # npv_model <- spec * (1 - prevalence) /
    #   ((1 - sens) * prevalence + spec * (1 - prevalence))

    # People in risk
    people_high_risk = np.sum(bernoulli.rvs(prob_model_high_risk, size=people))
    people_low_risk = people - people_high_risk

    # Probability that given High risk classification
    # the patient has a positive result with antigen

    prob_res_pos_given_high_risk_ag = sens * prob_res_pos_ag / prob_model_high_risk

    prob_res_neg_given_high_risk_ag = (
        (1 - spec) * prob_res_neg_ag / prob_model_high_risk
    )

    prob_res_pos_given_low_risk_ag = (
        1 - sens) * prob_res_pos_ag / prob_model_low_risk

    prob_res_pos_given_high_risk_lamp = sens * \
        prob_res_pos_lamp / prob_model_high_risk

    prob_res_neg_given_high_risk_lamp = 1 - prob_res_pos_given_high_risk_lamp

    prob_res_pos_given_high_risk_pcr = sens * \
        prob_res_pos_pcr / prob_model_high_risk

    prob_res_neg_given_high_risk_pcr = 1 - prob_res_pos_given_high_risk_pcr

    # Probability that given High risk classification
    # the patient has a negative result with antigen
    prob_res_neg_given_high_risk_ag = (
        (1 - spec) * prob_res_neg_ag / prob_model_high_risk
    )

    # PPV golden test
    ppv_golden_test = (
        sens_pcr_test
        * prevalence
        / (sens_pcr_test * prevalence + (1 - spec_pcr_test * (1 - prevalence)))
    )

    # Strategy 1 -------------------------------------------------------
    if strategy == 1:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            # Probability of people tested before of 5 days since symptoms onset.
            prob_symp_less_5d = 1 - prob_symp_more_5d

            #             people_low_risk = 0

            #             # Number of antigen tests done
            #             number_alt_tests = np.sum(
            #                 bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            #             )

            #             # Number of PCR tests done
            #             number_pcr_tests = (
            #                 people_high_risk
            #                 - number_alt_tests
            #                 + np.sum(
            #                     bernoulli.rvs(
            #                         prob_res_neg_given_high_risk_ag, size=number_alt_tests
            #                     )
            #                 )
            #             )

            #             number_test_per_person = (number_pcr_tests + number_alt_tests) / (
            #                 people_high_risk
            #             )

            #             number_positive_reported = (
            #                 people_res_pos_given_high_risk_ag
            #                 + people_res_pos_given_res_neg_ag_high_risk
            #                 + people_res_pos_given_high_risk_pcr
            #                 + pooling_tests_low_risk["pos_peop"]
            #             )

            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag,
                              size=people_low_risk)
            )

            number_alt_tests = people_symp_less_5d

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag,
                              size=people_symp_less_5d)
            )
            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            number_pcr_tests = (
                people_high_risk
                - number_alt_tests
                + np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_ag, size=number_alt_tests
                    )
                )
            )
            number_test_per_person = (number_pcr_tests + number_alt_tests) / (
                people_high_risk
            )
            # Total costs.
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
            )

            stock_capacity = ((number_pcr_tests + number_alt_tests) / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            efficiency = (number_positive_reported / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": [total_cost_alt],
                    "total_cost_pcr": [total_cost_pcr],
                    "total_cost": [total_cost],
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 2:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d

            pooling_tests_full_prevalence = pd.DataFrame()

            pooling_tests_full_prevalence = estimate_pooling_tests(
                group_size=group_size,
                p=prevalence,
                pop=people_low_risk,
                se=sens_pcr_test,
                s_loss=0,
                sp=spec_pcr_test,
            )

            pooling_tests_low_risk = estimate_pooling_tests(
                group_size=group_size,
                p=1 - npv_model,
                pop=people_low_risk,
                se=sens_pcr_test,
                s_loss=0,
                sp=spec_pcr_test,
            )

            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag,
                              size=people_low_risk)
            )

            number_alt_tests = people_symp_less_5d

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag,
                              size=people_symp_less_5d)
            )
            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            number_pcr_tests = (
                people_high_risk
                - number_alt_tests
                + np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_ag, size=number_alt_tests
                    )
                )
                + pooling_tests_low_risk["total_tests_pooling"].values
            )
            number_test_per_person = (
                number_pcr_tests + number_alt_tests) / (people)
            # Total costs.
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
                + pooling_tests_low_risk["pos_peop"]
            )

            # number_positive_reported <-
            #   people_high_risk * prob_symp_less_5d *
            #   (
            #     prob_res_pos_given_high_risk_ag +
            #       prob_res_neg_given_high_risk_ag *
            #         prob_res_pos_given_high_risk_pcr
            #   ) +
            #   people_high_risk * prob_symp_more_5d *
            #     prob_res_pos_given_high_risk_pcr +
            #   pooling_tests_low_risk$pos_peop
            stock_capacity = ((number_pcr_tests + number_alt_tests) / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            efficiency = (number_positive_reported / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 3:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d
            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag,
                              size=people_low_risk)
            )

            number_alt_tests = (
                people_symp_less_5d + people_low_risk + people_res_pos_given_low_risk_ag
            )

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag,
                              size=people_symp_less_5d)
            )

            number_pcr_tests = people_symp_more_5d + people_res_neg_given_high_risk_ag

            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            people_confirmed_retest_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag **
                              2, size=people_low_risk)
            )

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
                + people_confirmed_retest_ag
            )

            number_test_per_person = (
                number_pcr_tests + number_alt_tests) / (people)
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            stock_capacity = ((number_pcr_tests + number_alt_tests) / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            efficiency = (number_positive_reported / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 4:
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d
            prob_model_first_responder_pcr = 0.01
            people_first_responder = np.sum(
                bernoulli.rvs(prob_model_first_responder_pcr, size=people)
            )
            people_high_risk = np.sum(
                bernoulli.rvs(
                    prob_model_high_risk, size=people - people_first_responder
                )
            )
            people_low_risk = people - people_first_responder - people_high_risk

            # Number of test
            number_pcr_tests = people_first_responder

            number_lamp_tests = (
                np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_lamp, size=people_high_risk
                    )
                )
                + people_high_risk
            )

            number_ag_tests = (
                np.sum(
                    bernoulli.rvs(prob_res_pos_given_low_risk_ag,
                                  size=people_low_risk)
                )
                + people_low_risk
            )

            number_alt_tests = number_lamp_tests + number_ag_tests

            Highrisk = np.sum(
                bernoulli.rvs(prob_res_pos_given_high_risk_pcr,
                              size=people_high_risk)
            )

            number_positive_reported = (
                np.sum(bernoulli.rvs(prob_res_pos_pcr,
                       size=people_first_responder))
                + Highrisk
                + np.sum(
                    bernoulli.rvs(
                        1 - prob_res_pos_given_high_risk_pcr, size=Highrisk)
                )
                + np.sum(
                    bernoulli.rvs(
                        prob_res_pos_given_low_risk_ag**2, size=people_low_risk
                    )
                )
            )

            number_test_per_person = (
                number_pcr_tests + number_alt_tests) / (people)

            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost_alt = (
                cost_lamp * number_lamp_tests + cost_antigen * number_ag_tests
            )
            total_cost = total_cost_alt + total_cost_pcr

            stock_capacity = ((number_pcr_tests + number_alt_tests) / total_cost) * (
                (people_high_risk + people_low_risk +
                 people_first_responder) / people
            )

            efficiency = (number_positive_reported / total_cost) * (
                (people_high_risk + people_low_risk +
                 people_first_responder) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [
                        people_high_risk + people_low_risk + people_first_responder
                    ],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "people_first_responder": [people_first_responder],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "sens_lamp_test": [sens_lamp_test],
                    "spec_lamp_test": [spec_lamp_test],
                    "prob_model_first_responder_pcr,": [prob_model_first_responder_pcr],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )
            df_out = pd.concat([df_out, df2])

    return df_out


In [15]:
def r_covid_testing_strategies(
    prevalence,
    sens_model,
    spec_model,
    strategy,
    people,
    n_simul=1000,
    sens_ag_test=0.8,
    spec_ag_test=0.95,
    sens_pcr_test=0.95,
    spec_pcr_test=0.95,
    sens_lamp_test=0.95,
    spec_lamp_test=0.95,
    cost_pcr=100,
    cost_antigen=50,
    cost_lamp=20,
    group_size=5,
    budget=100000,
):
    resultados = pd.DataFrame()
    for s in range(n_simul):
        df = covid_testing_strategies(
            prevalence,
            sens_model,
            spec_model,
            sens_ag_test,
            spec_ag_test,
            sens_pcr_test,
            spec_pcr_test,
            sens_lamp_test,
            spec_lamp_test,
            people,
            cost_pcr,
            cost_antigen,
            cost_lamp,
            group_size,
            strategy,
            budget,
        )
        df["nsimul"] = s + 1
        resultados = pd.concat([resultados, df], ignore_index=True)
    return resultados


In [16]:
model_parameters = pd.read_pickle(
    filepath_or_buffer="data/model_parameters.pkl")


In [17]:
df_strategy_1 = r_covid_testing_strategies(
    strategy=1,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_1.to_pickle(path="data/df_strategy_1.pkl")
df_strategy_1.head(n=12)


,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,1,0.269096,1000,318,682,0.75,0.812287,0.8,0.95,0.75,...,177,260,269.096355,1.317610,12100,17700,29800,1406.040268,872.483221,1
1,1,0.269096,1000,318,682,0.75,0.812287,0.8,0.95,0.50,...,234,242,269.096355,1.223270,7750,23400,31150,1248.796148,776.886035,1
2,1,0.269096,1000,318,682,0.75,0.812287,0.8,0.95,0.25,...,270,220,269.096355,1.075472,3600,27000,30600,1117.647059,718.954248,1
3,1,0.269096,1000,350,650,0.75,0.812287,0.8,0.95,0.75,...,207,289,269.096355,1.351429,13300,20700,34000,1391.176471,850.000000,2
4,1,0.269096,1000,350,650,0.75,0.812287,0.8,0.95,0.50,...,248,257,269.096355,1.220000,8950,24800,33750,1265.185185,761.481481,2
5,1,0.269096,1000,350,650,0.75,0.812287,0.8,0.95,0.25,...,307,239,269.096355,1.140000,4600,30700,35300,1130.311615,677.053824,2
6,1,0.269096,1000,360,640,0.75,0.812287,0.8,0.95,0.75,...,197,278,269.096355,1.280556,13200,19700,32900,1401.215805,844.984802,3
7,1,0.269096,1000,360,640,0.75,0.812287,0.8,0.95,0.50,...,235,277,269.096355,1.205556,9950,23500,33450,1297.458894,828.101644,3
8,1,0.269096,1000,360,640,0.75,0.812287,0.8,0.95,0.25,...,303,258,269.096355,1.091667,4500,30300,34800,1129.310345,741.379310,3
9,1,0.269096,1000,367,633,0.75,0.812287,0.8,0.95,0.75,...,208,304,269.096355,1.280654,13100,20800,33900,1386.430678,896.755162,4


In [18]:
df_strategy_2 = r_covid_testing_strategies(
    strategy=2,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_2.to_pickle(path="data/df_strategy_2.pkl")
df_strategy_2.head(n=12)


,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,2,0.269096,1000,364,636,0.75,0.812287,0.8,0.95,0.75,...,368.423901,316.423901,269.096355,0.639424,13550,36842.390093,50392.390093,1268.889806,627.920010,1
1,2,0.269096,1000,364,636,0.75,0.812287,0.8,0.95,0.50,...,413.885972,296.885972,269.096355,0.594886,9050,41388.597207,50438.597207,1179.426084,588.608701,1
2,2,0.269096,1000,364,636,0.75,0.812287,0.8,0.95,0.25,...,476.936544,291.936544,269.096355,0.571937,4750,47693.654388,52443.654388,1090.573398,556.667050,1
3,2,0.269096,1000,332,668,0.75,0.812287,0.8,0.95,0.75,...,367.044130,290.044130,269.096355,0.604044,11850,36704.412965,48554.412965,1244.056086,597.358946,2
4,2,0.269096,1000,332,668,0.75,0.812287,0.8,0.95,0.50,...,399.936544,292.936544,269.096355,0.579937,9000,39993.654388,48993.654388,1183.697259,597.907112,2
5,2,0.269096,1000,332,668,0.75,0.812287,0.8,0.95,0.25,...,453.639072,271.639072,269.096355,0.538639,4250,45363.907247,49613.907247,1085.661465,547.505906,2
6,2,0.269096,1000,354,646,0.75,0.812287,0.8,0.95,0.75,...,359.126430,318.126430,269.096355,0.618126,12950,35912.642952,48862.642952,1265.028644,651.062673,3
7,2,0.269096,1000,354,646,0.75,0.812287,0.8,0.95,0.50,...,408.936544,299.936544,269.096355,0.585937,8850,40893.654388,49743.654388,1177.912140,602.964433,3
8,2,0.269096,1000,354,646,0.75,0.812287,0.8,0.95,0.25,...,462.151715,297.151715,269.096355,0.561152,4950,46215.171542,51165.171542,1096.745498,580.769509,3
9,2,0.269096,1000,332,668,0.75,0.812287,0.8,0.95,0.75,...,353.911258,289.911258,269.096355,0.602911,12450,35391.125798,47841.125798,1260.236351,605.987533,4


In [19]:
df_strategy_3 = r_covid_testing_strategies(
    strategy=3,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_3.to_pickle(path="data/df_strategy_3.pkl")
df_strategy_3.head(n=12)


,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,3,0.269096,1000,321,679,0.75,0.812287,0.8,0.95,0.75,...,181,273,269.096355,1.167,49300,18100,67400,1731.454006,405.044510,1
1,3,0.269096,1000,321,679,0.75,0.812287,0.8,0.95,0.50,...,225,248,269.096355,1.115,44500,22500,67000,1664.179104,370.149254,1
2,3,0.269096,1000,321,679,0.75,0.812287,0.8,0.95,0.25,...,271,235,269.096355,1.109,41900,27100,69000,1607.246377,340.579710,1
3,3,0.269096,1000,358,642,0.75,0.812287,0.8,0.95,0.75,...,202,285,269.096355,1.167,48250,20200,68450,1704.894083,416.362308,2
4,3,0.269096,1000,358,642,0.75,0.812287,0.8,0.95,0.50,...,245,268,269.096355,1.129,44200,24500,68700,1643.377001,390.101892,2
5,3,0.269096,1000,358,642,0.75,0.812287,0.8,0.95,0.25,...,300,256,269.096355,1.100,40000,30000,70000,1571.428571,365.714286,2
6,3,0.269096,1000,361,639,0.75,0.812287,0.8,0.95,0.75,...,201,288,269.096355,1.175,48700,20100,68800,1707.848837,418.604651,3
7,3,0.269096,1000,361,639,0.75,0.812287,0.8,0.95,0.50,...,253,271,269.096355,1.131,43900,25300,69200,1634.393064,391.618497,3
8,3,0.269096,1000,361,639,0.75,0.812287,0.8,0.95,0.25,...,324,251,269.096355,1.105,39050,32400,71450,1546.536039,351.294612,3
9,3,0.269096,1000,331,669,0.75,0.812287,0.8,0.95,0.75,...,182,273,269.096355,1.150,48400,18200,66600,1726.726727,409.909910,4


In [20]:
df_strategy_4 = r_covid_testing_strategies(
    strategy=4,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_4.to_pickle(path="data/df_strategy_4.pkl")
df_strategy_4.head(n=12)


,strategy,prevalence,people,people_high_risk,people_low_risk,people_first_responder,sens_model,spec_model,sens_ag_test,spec_ag_test,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,4,0.269096,1000,357,635,8,0.75,0.812287,0.8,0.95,...,8,300,269.096355,1.181,44010,800,44810,2635.572417,669.493417,1
1,4,0.269096,1000,321,664,15,0.75,0.812287,0.8,0.95,...,15,296,269.096355,1.173,44970,1500,46470,2524.209167,636.970088,1
2,4,0.269096,1000,329,662,9,0.75,0.812287,0.8,0.95,...,9,296,269.096355,1.185,45240,900,46140,2568.270481,641.525791,1
3,4,0.269096,1000,339,653,8,0.75,0.812287,0.8,0.95,...,8,305,269.096355,1.175,44520,800,45320,2592.674316,672.992056,2
4,4,0.269096,1000,345,641,14,0.75,0.812287,0.8,0.95,...,14,313,269.096355,1.175,43980,1400,45380,2589.246364,689.731159,2
5,4,0.269096,1000,355,630,15,0.75,0.812287,0.8,0.95,...,15,339,269.096355,1.191,44100,1500,45600,2611.842105,743.421053,2
6,4,0.269096,1000,318,675,7,0.75,0.812287,0.8,0.95,...,7,268,269.096355,1.184,45350,700,46050,2571.118350,581.976113,3
7,4,0.269096,1000,332,652,16,0.75,0.812287,0.8,0.95,...,16,298,269.096355,1.184,45110,1600,46710,2534.789124,637.979019,3
8,4,0.269096,1000,341,649,10,0.75,0.812287,0.8,0.95,...,10,320,269.096355,1.171,44400,1000,45400,2579.295154,704.845815,3
9,4,0.269096,1000,332,659,9,0.75,0.812287,0.8,0.95,...,9,315,269.096355,1.162,44450,900,45350,2562.293275,694.597574,4
